In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Churn essentially means if the customer exits
## Customer id, surname are and rowNuber are not useful to us so we drop these columns


In [3]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis = 1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [6]:
df["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
df["Gender"] = df["Gender"].map({"Male":1,"Female":0})

In [8]:
catagorical =["Geography"]
df = pd.get_dummies(data = df,prefix = catagorical,columns=catagorical).astype(int)

In [9]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0,1,1,1,101348,1,1,0,0
1,608,0,41,1,83807,1,0,1,112542,0,0,0,1
2,502,0,42,8,159660,3,1,0,113931,1,1,0,0
3,699,0,39,1,0,2,0,0,93826,0,1,0,0
4,850,0,43,2,125510,1,1,1,79084,0,0,0,1


In [10]:
df["Exited"].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [11]:
X = df.drop("Exited",axis = 1)
Y = df["Exited"]

## Adressing data imbalance using OverSampling

In [12]:
from imblearn.over_sampling import RandomOverSampler
oversampler = RandomOverSampler(random_state = 3)
X_new,Y_new = oversampler.fit_resample(X,Y)

## Using Logistic Regression 

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score
from xgboost import XGBClassifier


In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X_new,Y_new,test_size = 0.1,stratify=Y_new,random_state = 3)

In [15]:
model = LogisticRegression(max_iter = 1000)
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)

In [16]:
model.fit(X_train_scl,Y_train)

LogisticRegression(max_iter=1000)

In [17]:
pred_y_log_train = model.predict(X_train_scl)

In [18]:
print(accuracy_score(pred_y_log_train,Y_train))

0.6980394892904486


In [19]:
X_test_scl = scaler.transform(X_test)
pred_y_test_log = model.predict(X_test_scl)

In [20]:
print(accuracy_score(pred_y_test_log,Y_test))

0.7150031387319523


In [21]:
print(precision_score(pred_y_test_log,Y_test))

0.7110552763819096


## Prediction is about 72.3% accurate which is not that great 


In [22]:
xgmodel = XGBClassifier(n_estimators = 100)

In [23]:
xgmodel.fit(X_train_scl,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
pred_y_train_tree = xgmodel.predict(X_train_scl)

In [25]:
print(accuracy_score(pred_y_train_tree,Y_train))

0.9561152584943836


In [26]:
pred_y_test_tree = xgmodel.predict(X_test_scl)

In [27]:
print(accuracy_score(pred_y_test_tree,Y_test))

0.90646578782172


## achieves an accuracy of 90.6% only 


In [28]:
from sklearn.metrics import precision_score

In [29]:
print(precision_score(pred_y_test_tree,Y_test))

0.9371859296482412


## XGBoost ran much better with a precision score of 93.17% 